In [1]:
import pandas as pd
from os import listdir
import itertools

In [2]:
# Coletando nomes dos arquivo CSV presentes na pasta "path_to_dir"
path_to_dir = "data/BDTD_feb2025"
filenames = listdir(path_to_dir)
csv_files = [ path_to_dir + "/" + filename for filename in filenames if filename.endswith( ".csv" ) ]

In [3]:
# Concatenando todos os arquivos
data = pd.DataFrame()
for csv in csv_files:
    data = pd.concat([data, pd.read_csv(csv)])
data = data.drop_duplicates(subset=['Título'], keep='first').reset_index(drop=True)

In [4]:
# Quebrando assuntos em lista
data["Assuntos em português"] = data["Assuntos em português"].apply(lambda x: x.split("||"))
data["Assuntos em inglês"] = data["Assuntos em inglês"].apply(lambda x: x.split("||"))

In [5]:
# Limpando o nome dos autores
autors = data["Autor(a)"].apply(lambda x: x.split("||"))

new_autors = []
for n in range(len(autors)):
    new_autors.append(autors[n][0])
    if len(autors[n]) > 1:
        if autors[n][1].startswith("http://lattes.cnpq.br/"):
            data.loc[n, "ID Lattes do(a) autor(a)"] = autors[n][1]

data["Autor(a)"] = new_autors

In [6]:
# Limpando os nomes dos orientadores

orientadores = data["Orientadores"].apply(lambda x: x.split("||"))

for n in range(len(orientadores)):
    if "Não informado pela instituição" in orientadores[n]:
        orientadores[n] = list(filter(lambda a: a != "Não informado pela instituição", orientadores[n]))

data["Orientadores"] = orientadores

In [7]:
# Limpando os nomes dos membros da banca

banca = data["Membros da banca"].apply(lambda x: x.split("||"))

for n in range(len(banca)):
    if "Não informado pela instituição" in banca[n]:
        banca[n] = list(filter(lambda a: a != "Não informado pela instituição", banca[n]))

data["Membros da banca"] = banca

In [8]:
# Limpando os textos do Resumo
data['Resumo'] = data['Resumo'].astype(str)
data.loc[data['Resumo'].str.contains('Orientador:'), 'Resumo'] = ''
data.loc[data['Resumo'].str.contains('Orientadora:'), 'Resumo'] = ''
data.loc[data['Resumo'].str.contains('Orientadores:'), 'Resumo'] = ''
data.loc[data['Resumo'] == '--', 'Resumo'] = ''
data.loc[data['Resumo'] == '.', 'Resumo'] = ''
data.loc[data['Resumo'].str.contains(r'Dissertação \(mestrado\) -', regex=True), 'Resumo'] = ''
data.loc[data['Resumo'].str.contains('também disponível em formato'), 'Resumo'] = ''
data.loc[data['Resumo'] == '[sem resumo]', 'Resumo'] = ''
data.loc[data['Resumo'] == 'abstract', 'Resumo'] = ''
data.loc[data['Resumo'].str.contains('Tese \(doutorado\)'), 'Resumo'] = ''
data.loc[data['Resumo'] == 'Não informado pela instituição', 'Resumo'] = ''

Filtrar as áreas de conhecimento e assuntos indesejados

In [9]:
print("Total de documentos antes da limpeza: ", len(data))

Total de documentos antes da limpeza:  24240


In [10]:
# Criando um DataFrame com os assuntos e áreas do conhecimento preprocessados
AC_assunt = data[['Área do conhecimento CNPq', 'Assuntos em português', 'Assuntos em inglês']].copy()

AC_assunt.loc[:, 'Área do conhecimento CNPq'] = AC_assunt['Área do conhecimento CNPq'].apply(lambda x: x.split('||'))
AC_assunt.loc[:, 'Área do conhecimento CNPq'] = AC_assunt['Área do conhecimento CNPq'].apply(lambda x: [a.split('::') if '::' in a else a for a in x])
AC_assunt.loc[:, 'Área do conhecimento CNPq'] = AC_assunt['Área do conhecimento CNPq'].apply(lambda x: list(set([item.lower() for sublist in x for item in (sublist if isinstance(sublist, list) else [sublist])])))
AC_assunt.loc[:, 'Assuntos em português'] = AC_assunt['Assuntos em português'].apply(lambda x: [a.lower() for a in x])
AC_assunt.loc[:, 'Assuntos em inglês'] = AC_assunt['Assuntos em inglês'].apply(lambda x: [a.lower() for a in x])

In [11]:
# Funcao para limpar a base de dados de acordo com a area do conhecimento e lista de excessoes
def cleanAreaAssunto(area, lista_excessoes):
    
  indices_to_remove =  AC_assunt[AC_assunt.apply(lambda x : area in x['Área do conhecimento CNPq'] and 
                                                all(elem not in x['Assuntos em português'] for elem in lista_excessoes),
                                                axis=1)].index
  # Remove the rows with the specified indices
  return data.drop(indices_to_remove)

In [12]:
area = 'medicina veterinária'
lista_excessoes = []
data = cleanAreaAssunto(area, lista_excessoes)

In [13]:
area = 'agronomia'
lista_excessoes = ['terras indígenas', 'conhecimento local indígena', 'índios da américa do sul - alimentos - amazônia', 'affirmative action',
                   'feira de agricultura familiar', 'local knowledge', 'conflitos fundiários', 'conhecimento tradicional associado',
                   'memória', 'quilombolas peoples', 'comunidade quilombola de itacoã-miri - acará (pa)', 'quilombo', 'comunidades tradicionais populares',
                   'guarani m’bya', 'community quilombola','xavante', 'mulheres quilombolas', 'traditional popular communities', 'conhecimento ecológico tradicional',
                   'negro (raça)', 'índios tikúna - alimentos', 'quilombolas - amazônia - alimentos', 'comunidades tradicionais', 'quilombolas - ilha do marajó',
                   'quilombolas - mato grosso', 'povos quilombolas', 'quilombolas - marajó, ilha do (pa)', 'terra preta de índio', 'índios tukúna – amazônia']
data = cleanAreaAssunto(area, lista_excessoes)

In [14]:
print("Total de documentos após a limpeza: ", len(data))

Total de documentos após a limpeza:  23968


In [ ]:
# Area do conheciumento CNPq
lista_excessoes = ['terras indígenas', 'conhecimento local indígena', 'índios da américa do sul - alimentos - amazônia', 'affirmative action',
                   'feira de agricultura familiar', 'local knowledge', 'conflitos fundiários', 'conhecimento tradicional associado',
                   'memória', 'quilombolas peoples', 'comunidade quilombola de itacoã-miri - acará (pa)', 'quilombo', 'comunidades tradicionais populares',
                   'guarani m’bya', 'community quilombola','xavante', 'mulheres quilombolas', 'traditional popular communities', 'conhecimento ecológico tradicional'c,
                   'negro (raça)', 'índios tikúna - alimentos', 'quilombolas - amazônia - alimentos', 'comunidades tradicionais', 'quilombolas - ilha do marajó',
                   'quilombolas - mato grosso', 'povos quilombolas', 'quilombolas - marajó, ilha do (pa)', 'terra preta de índio', 'índios tukúna – amazônia']
#df = AC_assunt[AC_assunt['Área do conhecimento CNPq'].apply(lambda x : 'agronomia' in x)]
df = AC_assunt[AC_assunt.apply(lambda x : 'agronomia' in x['Área do conhecimento CNPq'] and 
                     all(elem not in x['Assuntos em português'] for elem in lista_excessoes),
                       axis=1)]

print(set(itertools.chain.from_iterable(df['Assuntos em português'])))

# Assuntos em português 4734
#data[AC_assunt['Assuntos em português'].apply(lambda x : 'slave' in x)][['Título', 'Assuntos em português', 'Assuntos em inglês']]

# Assuntos em inglês
#data[AC_assunt['Assuntos em inglês'].apply(lambda x : 'slave' in x)][['Título', 'Assuntos em português', 'Assuntos em inglês']]

{'lesion nematode', 'potássio trocável', 'híbridos de milho', 'agricultura empresarial', 'solubilização', 'produtos químicos agrícolas', 'espécies adaptadas', 'experimento longa duração', 'índios da américa do sul - alimentos - amazônia', 'variância genética', 'biological activity', 'genótipos', 'tulipa', 'inoculação com raízes noduladas', 'biologia do solo', 'polinifagia', 'fixação biológica de nitrogênio', 'affirmative action', 'feira de agricultura familiar', 'termitidae', 'gerbera jamesonii bolus ex. hook', 'agrobiodiversidade', 'citrus spp.', 'qualidade dos frutos', 'fbn', 'segurança', 'food habits', 'unidade de conservação', 'plantios de mudas', 'sementes', 'fertilizante organomineral', 'cadeias produtivas', 'k-means', 'vigna radiata', 'campo nativo', 'óxido nitroso', 'seletividade de defensivos alternativos', 'local knowledge', 'desenvolvimento territorial', 'rurality', 'híbrido', 'práticas tradicionais', 'pcr em tempo real', 'musa spp', 'concepções discentes', 'oryza sativa l.'

In [15]:
# Reset the index if needed
data = data.reset_index(drop=True)

# Salvando como csv
path_to_dir_output = "data"
data.to_csv(path_to_dir_output + "/BDTD_Consolidado_feb2025.csv", index=False)